In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import scikit.learn as skl
%matplotlib inline
plt.style.use("fivethirtyeight")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [13]:
#data = pd.read_csv('C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/advertising.csv', index_col=0)
url = 'C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/bikeshare.csv'
df = pd.read_csv(url, index_col='datetime', parse_dates=True)
df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
datetime,,,,,,,,,
2011-01-01 00:00:00,Spring,0,0,Clear Skies,9.84,14.395,81,0.0,16
2011-01-01 01:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,40
2011-01-01 02:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,32
2011-01-01 03:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,13
2011-01-01 04:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,1


In [14]:
df.weather.value_counts()

Clear Skies          7192
Partly Cloudy        2834
Light Storms/Rain     859
Heavy Storms/Rain       1
Name: weather, dtype: int64

In [16]:
#convert categorical vaiables to rank variable

##?df.season.map

weather_map = {
    'Clear Skies':       4,
    'Partly Cloudy':     3,
    'Light Storms/Rain': 2,
    'Heavy Storms/Rain': 1
}

df.weather = df.weather.map(weather_map)

In [19]:
df.weather.value_counts()

4    7192
3    2834
2     859
1       1
Name: weather, dtype: int64

In [25]:
#One hot coding (creating dummy variables)

pd.get_dummies(df.season).head()

,Fall,Spring,Summer,Winter
datetime,,,,
2011-01-01 00:00:00,0,1,0,0
2011-01-01 01:00:00,0,1,0,0
2011-01-01 02:00:00,0,1,0,0
2011-01-01 03:00:00,0,1,0,0
2011-01-01 04:00:00,0,1,0,0


In [24]:
#use below to ajust for multicollinearity
pd.get_dummies(df.season, drop_first=True).head()

,Spring,Summer,Winter
datetime,,,
2011-01-01 00:00:00,1,0,0
2011-01-01 01:00:00,1,0,0
2011-01-01 02:00:00,1,0,0
2011-01-01 03:00:00,1,0,0
2011-01-01 04:00:00,1,0,0


In [26]:
#convert an entire dataframe
df = pd.get_dummies(df, drop_first=True)
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,season_Spring,season_Summer,season_Winter
datetime,,,,,,,,,,,
2011-01-01 00:00:00,0,0,4,9.84,14.395,81,0.0,16,1,0,0
2011-01-01 01:00:00,0,0,4,9.02,13.635,80,0.0,40,1,0,0
2011-01-01 02:00:00,0,0,4,9.02,13.635,80,0.0,32,1,0,0
2011-01-01 03:00:00,0,0,4,9.84,14.395,75,0.0,13,1,0,0
2011-01-01 04:00:00,0,0,4,9.84,14.395,75,0.0,1,1,0,0


In [63]:
#using all the variables

X = df.loc[:, df.columns != 'count']
y = df['count']

X_std = (X - X.mean()) / X.std()

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_std, y)
print(lr.score(X_std, y))
print(list(zip(X, lr.coef_)))


0.2752503232099429
[('holiday', -1.4378267724123381), ('workingday', -1.3339375283802792), ('weather', -1.680745934591319), ('temp', 62.03831630929895), ('atemp', 24.65228381735198), ('humidity', -54.25768583931854), ('windspeed', 4.420026574191252), ('season_Spring', 16.284212733988465), ('season_Summer', 15.168533059848818), ('season_Winter', 44.66141566782872)]


In [ ]:
#grab all categorical variables
#num_cols = X.select_dtypes(include=np.object).columns.tolist()

In [57]:
#Another way
#grab all numeric columns
num_cols = X.select_dtypes(include=np.number).columns.tolist()

X_std = (df[num_cols] - df[num_cols].mean()) / df[num_cols].std()


from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_std, y)
print(lr.score(X_std, y))
print(list(zip(X, lr.coef_)))


0.2752503232099429
[('holiday', -1.4378267724123381), ('workingday', -1.3339375283802792), ('weather', -1.680745934591319), ('temp', 62.03831630929895), ('atemp', 24.65228381735198), ('humidity', -54.25768583931854), ('windspeed', 4.420026574191252), ('season_Spring', 16.284212733988465), ('season_Summer', 15.168533059848818), ('season_Winter', 44.66141566782872)]


In [60]:
#rank coeffecients

coeffs = pd.DataFrame({
    'Variable': X_std.columns,
    'Weight'  : lr.coef_
}).sort_values(by='Weight', ascending=False)
coeffs

,Variable,Weight
3,temp,62.038316
9,season_Winter,44.661416
4,atemp,24.652284
7,season_Spring,16.284213
8,season_Summer,15.168533
6,windspeed,4.420027
1,workingday,-1.333938
0,holiday,-1.437827
2,weather,-1.680746
5,humidity,-54.257686


In [73]:
#using some of the variables 

feature_cols = ['weather', 'temp', 'atemp', 'humidity']

X = df[feature_cols]
y = df['count']

X_std = (X - X.mean()) / X.std()

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_std, y)
print(lr.score(X_std, y))
print(list(zip(feature_cols, lr.coef_)))


0.24307517355292296
[('weather', -2.9105767358416523), ('temp', 23.16714272625698), ('atemp', 45.54751919856181), ('humidity', -55.18486553237926)]


In [74]:
#Another package, don't use this package

import statsmodels.api as sm

mod = sm.OLS(y, X)
results = mod.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     4739.
Date:                Wed, 31 Jul 2019   Prob (F-statistic):               0.00
Time:                        19:53:42   Log-Likelihood:                -70642.
No. Observations:               10886   AIC:                         1.413e+05
Df Residuals:                   10882   BIC:                         1.413e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
weather       29.1045      1.359     21.418      0.0

In [76]:
#data
url = 'C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/sac_re_cat.csv'
prices = pd.read_csv(url, index_col='sale_date', parse_dates=True)
prices.head()

,street,city,zip,state,beds,baths,sq__ft,type,price
sale_date,,,,,,,,,
2008-05-21,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,59222
2008-05-21,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,68212
2008-05-21,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,68880
2008-05-21,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,69307
2008-05-21,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,81900


In [114]:
#another way to re-index by date
prices = pd.read_csv('C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/sac_re_cat.csv')
prices = prices.set_index('sale_date')
prices.head()

,street,city,zip,state,beds,baths,sq__ft,type,price
sale_date,,,,,,,,,
Wed May 21 00:00:00 EDT 2008,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,59222
Wed May 21 00:00:00 EDT 2008,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,68212
Wed May 21 00:00:00 EDT 2008,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,68880
Wed May 21 00:00:00 EDT 2008,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,69307
Wed May 21 00:00:00 EDT 2008,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,81900


In [ ]:
#prices
#pd.get_dummies(df.season, drop_first=True)

In [89]:
prices.zip.value_counts()

95648    72
95823    61
95828    45
95758    44
95838    37
95835    37
95757    36
95624    34
95843    33
95621    28
95822    24
95820    23
95762    23
95842    22
95834    22
95632    21
95660    21
95670    21
95608    20
95678    20
95833    20
95747    20
95815    18
95826    18
95630    17
95825    13
95673    13
95824    12
95832    12
95742    11
         ..
95661     8
95610     7
95817     7
95818     7
95841     7
95677     6
95821     6
95864     5
95603     5
95693     5
95819     4
95816     4
95683     4
95626     4
95726     3
95691     3
95814     3
95746     3
95623     2
95650     2
95811     2
95614     1
95619     1
95663     1
95631     1
95633     1
95635     1
95655     1
95690     1
95722     1
Name: zip, Length: 68, dtype: int64

In [87]:
prices.zip = prices.zip.astype('category')

In [88]:
#index is zip
prices.groupby('zip').zip.count()

zip
95603     5
95608    20
95610     7
95614     1
95619     1
95621    28
95623     2
95624    34
95626     4
95628     9
95630    17
95631     1
95632    21
95633     1
95635     1
95648    72
95650     2
95655     1
95660    21
95661     8
95662    11
95663     1
95667    10
95670    21
95673    13
95677     6
95678    20
95682    10
95683     4
95690     1
         ..
95758    44
95762    23
95765    11
95811     2
95814     3
95815    18
95816     4
95817     7
95818     7
95819     4
95820    23
95821     6
95822    24
95823    61
95824    12
95825    13
95826    18
95827     9
95828    45
95829    11
95831    10
95832    12
95833    20
95834    22
95835    37
95838    37
95841     7
95842    22
95843    33
95864     5
Name: zip, Length: 68, dtype: int64

In [91]:
#index is now the real index
transformed = prices.groupby('zip').zip.transform('count')

In [118]:
#Group Categorical columns into buckets

prices['zip'] = np.where(transformed < 25, 'Other', prices['zip'])
prices.zip.value_counts()

Other    558
95648     72
95823     61
95828     45
95758     44
95835     37
95838     37
95757     36
95624     34
95843     33
95621     28
Name: zip, dtype: int64

In [109]:
#Regression here
X = prices.loc[:, ['zip', 'beds', 'baths', 'sq__ft']]
y = prices['price']

#standardize
X = pd.get_dummies(X, drop_first=True)
num_cols = X.select_dtypes(include=np.number).columns.tolist()
X[num_cols]= (X[num_cols] - X[num_cols].mean()) / X[num_cols].std()


# Instantiate and fit.
lr = LinearRegression()
lr.fit(X, y)

# Print the coefficients.
##print(lr.intercept_)
##print(lr.coef_)
print(lr.score(X,y))
list(zip(X, lr.coef_))

0.27115491268070757


[('beds', -11298.894188546952),
 ('baths', 54575.06874890915),
 ('sq__ft', 11079.698439309122),
 ('zip_95624', 10091.517352691468),
 ('zip_95648', -4654.107719768895),
 ('zip_95757', 20311.135206179842),
 ('zip_95758', 7630.416201122458),
 ('zip_95823', -3445.5794225572195),
 ('zip_95828', -2509.029671760694),
 ('zip_95835', 11924.245380295413),
 ('zip_95838', -4414.217140615041),
 ('zip_95843', 4441.244516933082),
 ('zip_Other', 40848.16474900619)]

In [117]:
#Regression here
X = prices.loc[:, ['zip', 'beds', 'baths', 'sq__ft']]
y = prices['price']

#standardize
num_cols = X.select_dtypes(include=np.number).columns.tolist()
X[num_cols]= (X[num_cols] - X[num_cols].mean()) / X[num_cols].std()
X = pd.get_dummies(X, drop_first=True)

# Instantiate and fit.
lr = LinearRegression()
lr.fit(X, y)

# Print the coefficients.
##print(lr.intercept_)
##print(lr.coef_)
print(lr.score(X,y))
list(zip(X, lr.coef_))

0.19945730163613828


[('zip', -18600.15053279643),
 ('beds', -6663.566341394699),
 ('baths', 51675.33458830176),
 ('sq__ft', 19790.44364634824)]

In [120]:
#Regression here
X = prices.loc[:, ['zip','type', 'beds', 'baths', 'sq__ft']]
y = prices['price']

#standardize
num_cols = X.select_dtypes(include=np.number).columns.tolist()
X[num_cols]= (X[num_cols] - X[num_cols].mean()) / X[num_cols].std()
X = pd.get_dummies(X, drop_first=True)

# Instantiate and fit.
lr = LinearRegression()
lr.fit(X, y)

# Print the coefficients.
##print(lr.intercept_)
##print(lr.coef_)
print(lr.score(X,y))
list(zip(X, lr.coef_))

0.2953326582199547


[('beds', -19030.52250757088),
 ('baths', 60314.02509089212),
 ('sq__ft', 9607.171745785987),
 ('zip_95624', 52338.373514280174),
 ('zip_95648', -29362.999753053904),
 ('zip_95757', 101704.56622942087),
 ('zip_95758', 39539.35663826284),
 ('zip_95823', -19594.005777140854),
 ('zip_95828', -9993.291110779319),
 ('zip_95835', 64967.92148287481),
 ('zip_95838', -30069.343277734475),
 ('zip_95843', 20273.98385622061),
 ('zip_Other', 82152.23767199068),
 ('type_Multi-Family', -3224.8969339318423),
 ('type_Residential', 86881.99084533357),
 ('type_Unkown', 184209.92015020273)]

In [121]:
prices.head()

,street,city,zip,state,beds,baths,sq__ft,type,price
sale_date,,,,,,,,,
Wed May 21 00:00:00 EDT 2008,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,59222
Wed May 21 00:00:00 EDT 2008,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,68212
Wed May 21 00:00:00 EDT 2008,2796 BRANCH ST,SACRAMENTO,Other,CA,2,1,796,Residential,68880
Wed May 21 00:00:00 EDT 2008,2805 JANETTE WAY,SACRAMENTO,Other,CA,2,1,852,Residential,69307
Wed May 21 00:00:00 EDT 2008,6001 MCMAHON DR,SACRAMENTO,Other,CA,2,1,797,Residential,81900


In [124]:
prices['prediction'] = lr.predict(X)
prices.head()

,street,city,zip,state,beds,baths,sq__ft,type,price,prediction
sale_date,,,,,,,,,,
Wed May 21 00:00:00 EDT 2008,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,59222,113093.695557
Wed May 21 00:00:00 EDT 2008,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,68212,112733.338098
Wed May 21 00:00:00 EDT 2008,2796 BRANCH ST,SACRAMENTO,Other,CA,2,1,796,Residential,68880,224866.407864
Wed May 21 00:00:00 EDT 2008,2805 JANETTE WAY,SACRAMENTO,Other,CA,2,1,852,Residential,69307,225494.823964
Wed May 21 00:00:00 EDT 2008,6001 MCMAHON DR,SACRAMENTO,Other,CA,2,1,797,Residential,81900,224877.629580


In [132]:
#mean absolute error 
mae = np.mean(np.abs(prices['price'] - prices['prediction']))
print(mae)

#mean squared error
mse = np.mean((prices['price'] - prices['prediction'])**2)
print(mse)

86416.30313899217
13618442790.266226


In [139]:
#metrics
#https://scikit-learn.org/stable/modules/classes.html
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(prices['price'], prices['prediction']))
print('MSE:', metrics.mean_squared_error(prices['price'], prices['prediction']))


MAE: 86416.30313899208
MSE: 13618442790.26624


In [133]:
#preprocessing (automatically set up your data)
#https://scikit-learn.org/stable/modules/classes.html
from sklearn import preprocessing

In [137]:
#data = pd.read_csv('C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/advertising.csv', index_col=0)
url = 'C:/Users/James.Kelly/DAT-06-24/class material/Unit 3/data/bikeshare.csv'
df = pd.read_csv(url, index_col='datetime', parse_dates=True)
df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
datetime,,,,,,,,,
2011-01-01 00:00:00,Spring,0,0,Clear Skies,9.84,14.395,81,0.0,16
2011-01-01 01:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,40
2011-01-01 02:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,32
2011-01-01 03:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,13
2011-01-01 04:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,1


In [138]:
#https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DatetimeIndex.html
df['Hour'] = df.index.hour
df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,Hour
datetime,,,,,,,,,,
2011-01-01 00:00:00,Spring,0,0,Clear Skies,9.84,14.395,81,0.0,16,0
2011-01-01 01:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,40,1
2011-01-01 02:00:00,Spring,0,0,Clear Skies,9.02,13.635,80,0.0,32,2
2011-01-01 03:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,13,3
2011-01-01 04:00:00,Spring,0,0,Clear Skies,9.84,14.395,75,0.0,1,4


In [135]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=2)

In [136]:
X_train.shape

(788, 3)